### 1.初始化项目

当第一次使用Django搭建后端项目时，`cd`到自定义的目录下进行项目的初始化。初始化的内容会涉及到以下几方面的内容：

1. 数据库的配置
2. API的设置

```bash
#创建App Django项目
django-admin startproject cottonApp
```

初始化后的目录结构

```bash
cottonApp/
    manage.py
    cottonApp/
        __init__.py
        settings.py
        urls.py
        asgi.py
        wsgi.py
```

创建一个应用程序，一个项目中可以包含多个应用程序，应用程序其实就是一系列API的集合

> 这里创建一个叫polls的应用程序
```bash
python manage.py startapp polls
```

给polls应用设置对应的响应函数

> polls/views.py
```python
from django.http import HttpResponse


def index(request):
    return HttpResponse("Hello, world. You're at the polls index.")

```
配置路由,并且挂载到项目根目录上组成对应的API接口

> 在polls目录中创建子路由文件`urls.py`，并为其配置响应函数

```python
from django.urls import path

from . import views

urlpatterns = [
    path('', views.index, name='index'),
]
```
> 将polls路由挂载到项目根路由`cottonApp/urls.py`上
```python
from django.contrib import admin
from django.urls import include, path

urlpatterns = [
    path('polls/', include('polls.urls')), #挂载polls中的路由
    path('admin/', admin.site.urls),
]
```

### `path()`API
+ 路由，字符串类型
+ view模块中的`HttpResponse`响应函数
+ name 在使用模块的时候尤其有用，通过对单个文件的修改而改变全局的URL模式


## 2.个性化配置

1. 数据库引擎的配置
Django中默认使用的数据库引擎为`SQLite`，如果想要更改需要下载相应的组件以及修改配置文件
```python
##修改条目
 DATABASES 'default'
```

2.设置时区
` TIME_ZONE`

3.`INSTALLED APPS`包含在Django实例中的一些API，也可以自行添加自定义的API，之后可以打包给其他人用。而这些API的使用需要用到一些数据，在刚开始学习的时候可以使用Django中自带的一些数据来进行测试
```bash
##生成自定义数据
python manage.py migrate
```
通过`models.py`自定义数据表
```python
from django.db import models

#Question Table
# question_text column
# pub_data column
class Question(models.Model):
    question_text = models.CharField(max_length=200)
    pub_date = models.DateTimeField('date published')

#Choice Table
class Choice(models.Model):
    question = models.ForeignKey(Question, on_delete=models.CASCADE)
    choice_text = models.CharField(max_length=200)
    votes = models.IntegerField(default=0)

```
在`settings.py`中引入APP，并且生成对应的sql语句
```python
##引入APP
INSTALLED_APPS = [
    'polls.apps.PollsConfig',
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
]
##生成SQL语句

python manage.py makemigrations polls
##查看对应的sql语句
python manage.py sqlmigrate polls 0001
##执行生成的SQL语句
python manage.py migrate
```

#### `admin.py`文件

修改改文件可以让其能够在管理员界面显示
```python
from django.contrib import admin

from .models import Question
from .models import Choice
# Register your models here.

admin.site.register(Question)
admin.site.register(Choice)

```


## 3.创建APP



#### 3.1路由的配置
> `views`文件主要做了两件事情，针对客户端的请求，返回一个HttpResponse响应对象，或者是返回一个错误例如404代码；view中还能干好多事情，从数据库中获取数据，产生pdf文件或者是创建水印文件。

1. 调用`models.py`中数据表，直接操作数据库，例如提取数据库中前五项数据

 > 然鹅这样的后端还有弊端，就是数据与html高度耦合；不方便修改，这就需要我们的模板系统出马了

```python

def index(request):
    latest_question_list = Question.objects.order_by('-pub_date')[:5]
    output = "</br>".join(q.question_text for q in latest_question_list)
    return HttpResponse(output)
```

2. `templates`系统将数据与html页面解耦合

```python
from django.template import loader
from .models import Question
def index(request):
    latest_question_list = Question.objects.order_by('-pub_date')[:5]
    template = loader.get_template('polls/index.html')
    context = {
        'latest_question_list': latest_question_list,
    }
    return HttpResponse(template.render(context, request))
```

3. 处理404请求,通过抛出错误，返回Http404
> 还可以通过`get_object_or_404`或者` get_list_or_404()`函数实现

```python
from django.http import Http404
from django.shortcuts import render

from .models import Question
def detail(request, question_id):
    try:
        question = Question.objects.get(pk=question_id)
    except Question.DoesNotExist:
        raise Http404("Question does not exist")
    return render(request, 'polls/detail.html', {'question': question})

```
4. URL的命名空间
当模板文件中涉及到不同APP中的view响应函数时，使用命名空间对其进行区分

```python
<li><a href="{% url 'detail' question.id %}">{{ question.question_text }}</a></li>
```

5. 重定向`HttpResponseRedirect`
函数接收参数：
1. 重定向的路径，可以是绝对路径也可以是相对路径
2. 配合 `reverse()`函数可以解耦合，



## 4.进行模块测试
例如对Question代码进行测试，将所有的测试代码在`test.py`中封装成`QuestionModelTests`类
1. 在类中命名以`test`开头的测试方法
```python
# 测试polls app
python manage.py  test polls
```
### 对`view`进行测试
使用`client`模拟浏览器访问
```python
# 进入shell环境
python manage.py shell
from django.test.utils import setup_test_environment
setup_test_environment()
from django.test import Client
client = Client()
```
测序代码
```python
>>> # get a response from '/'
>>> response = client.get('/')
Not Found: /
>>> # we should expect a 404 from that address; if you instead see an
>>> # "Invalid HTTP_HOST header" error and a 400 response, you probably
>>> # omitted the setup_test_environment() call described earlier.
>>> response.status_code
404
>>> # on the other hand we should expect to find something at '/polls/'
>>> # we'll use 'reverse()' rather than a hardcoded URL
>>> from django.urls import reverse
>>> response = client.get(reverse('polls:index'))
>>> response.status_code
200
>>> response.content
b'\n    <ul>\n    \n        <li><a href="/polls/1/">What&#x27;s up?</a></li>\n    \n    </ul>\n\n'
>>> response.context['latest_question_list']
<QuerySet [<Question: What's up?>]>
```


## 5.静态文件夹
文件的结构和模板文件类似 `/polls/static/polls/style.css`



## 6.编写健壮的Django程序
将`polls`包进行封装并且实现分享
健壮的程序就最重要的特点就是：**代码可重用性**

1. 首先打包我们的程序
> 我们使用`setuptools`来打包我们的polls App
```bash
# 安装打包工具
 pip install setuptools --user
 在项目外创建一个目录名,分别创建对应的打包配置文件
 ```bash
 ## 进行打包
 python setup.py sdist
 ```



